# CAPSTONE PROJECT

#### THIS NOTEBOOK WILL BE MAINLY USED FOR THE CAPSTONE PROJECT FROM IBM DATA SCIENCE PROFESSIONAL CERTIFICATE

## Import important libraries

In [1]:
import numpy as np
import pandas as pd
print("Hello Capstone Project Course!")

Hello Capstone Project Course!
